In [1]:
import numpy as np 
import pandas as pd
from scipy import io
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

In [2]:
#df = pd.read_csv('full_df')

In [1]:
#df.head(2)

In [4]:
endpoints = ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD', 'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']

In [5]:
original_columns = ['ID',
 'inchikey',
 'sdftitle',
 'order',
 'set',
 'CVfold',
 'NR.AhR',
 'NR.AR',
 'NR.AR.LBD',
 'NR.Aromatase',
 'NR.ER',
 'NR.ER.LBD',
 'NR.PPAR.gamma',
 'SR.ARE',
 'SR.ATAD5',
 'SR.HSE',
 'SR.MMP',
 'SR.p53']

In [6]:
desired_column_order = ['ID', 'inchikey', 'inchi', 'SMILES', 'sdftitle', 'order', 'set', 'CVfold'] + [col for col in df.columns if col not in ['ID', 'inchikey', 'inshi', 'SMILES', 'sdftitle', 'order', 'set', 'CVfold']]
df_ordered = df[desired_column_order]

In [2]:
#df_ordered.head(2)
#df_ordered.shape

In [8]:
column_names = df_ordered.columns.tolist()

In [9]:
# drop columns with nan
#clean_df_NRAhR = filtered_df_NRAhR.dropna(axis=1).copy()

# list whatever didnt have nan
#clean_NRAhR_columns = clean_df_NRAhR.columns.tolist()

In [10]:
def inchikey_to_inchi(inchikey):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/property/InChI/TXT"
    try:
        response = requests.get(url, timeout=10)  # Adding a timeout for the request
        if response.status_code == 200:
            return response.text.strip()
        else:
            # Log the error but don't break the process
            print(f"Non-successful response for InChIKey {inchikey}: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        # Handle request exceptions, like timeouts, connection errors, etc.
        print(f"Request failed for InChIKey {inchikey}: {e}")
        return None

In [11]:
def inchi_to_smiles(inchi):
    mol = Chem.MolFromInchi(inchi)
    if mol is None:  # Check if RDKit could parse the InChI string
        return None
    smiles = Chem.MolToSmiles(mol)
    return smiles

In [12]:
def getMolDescriptorsFromInchiKey(inchikey, missingVal=None):
    inchi = inchikey_to_inchi(inchikey)
    if inchi is None:
        return None
    smiles = inchi_to_smiles(inchi)
    if smiles is None:
        return None
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    return getMolDescriptors(mol, missingVal)

In [13]:
def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm, fn in Descriptors._descList:
        try:
            val = fn(mol)
        except:
            val = missingVal
        res[nm] = val
    return res

In [14]:
def getMolDescriptorsFromSmiles(smiles, missingVal=None):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    res = {}
    for nm, fn in Descriptors._descList:
        try:
            val = fn(mol)
        except Exception as e:
            print(f"Error calculating descriptor {nm} for SMILES {smiles}: {e}")
            val = missingVal
        res[nm] = val
    return res

In [15]:
def make_feature_lists(full_df, target_endpoints):
    """
    input is a dataframe and a list of column names
    outputs a list, named feature_[target_endpoint], with all the column titles the model was actually trained with
    """
    for endpoint in target_endpoints:
        try:
            # Generate molecular descriptors from the SMILES string
            descriptors = getMolDescriptorsFromSmiles(smiles)
            
            # Filter rows with NaN in the target endpoint
            filtered_df = full_df.dropna(subset=[target_endpoint])
            
            # Drop columns with NaN values
            clean_df = filtered_df.dropna(axis=1).copy()
            
            # Define features excluding specific columns
            excluded_columns = ['ID', 'inchikey', 'inchi', 'SMILES', 'sdftitle', 'order', 'set', 'inchi', target_endpoint]
            features = clean_df.columns.difference(excluded_columns)
            
            # Select descriptor values for the current SMILES string
            descriptor_values = [descriptors[feature] for feature in features if feature in descriptors]
            
            # Reshape the descriptor values for prediction
            descriptor_values_2d = np.array(descriptor_values).reshape(1, -1)
            
            # Predict the endpoint
            prediction = test_sratad5_model.predict(descriptor_values_2d)
            
            # Print the SMILES string and its predicted endpoint
            print(f'SMILES: {smiles}, Predicted Endpoint: {prediction[0]}')
        
        except Exception as e:
            # Handle errors, such as missing descriptors or prediction issues
            print(f'Error processing SMILES: {smiles}. Error: {e}')

In [16]:
def process_endpoint(endpoint, full_df):
    # Step 1: Filter rows with NaN in the target endpoint
    filtered_df = df_ordered.dropna(subset=[endpoint])

    # Step 2: Drop columns with NaN values
    clean_df = filtered_df.dropna(axis=1).copy()

    # Step 3: Define features excluding specific columns
    features = clean_df.columns.difference(['ID', 'inchikey', 'inchi', 'SMILES', 'sdftitle', 'order', 'set', 'inchi', endpoint])

    # Step 4 & 5: Define X and Y
    X = clean_df[features].values
    y = clean_df[endpoint].values

    # Step 6: Check shapes and assert they match
    assert X.shape[0] == y.shape[0], f"Shape mismatch between X and y for endpoint {endpoint}"

    # Step 7: Split into test/train
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 8: Parameter grid
    param_grid = {
        'svc__C': [0.1, 1, 10],
        'svc__kernel': ['linear', 'rbf']
    }

    # Step 9: Create and train the SVM pipeline
    svm_pipeline = make_pipeline(StandardScaler(), SVC(probability=True))
    grid_search = GridSearchCV(svm_pipeline, param_grid, cv=4, scoring='accuracy', verbose=2)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Step 10: Evaluate the model
    predictions = best_model.predict(X_test)
    print(f"Evaluation metrics for endpoint {endpoint}:")
    print(f"Precision: {precision_score(y_test, predictions)}")
    print(f"Recall: {recall_score(y_test, predictions)}")
    print(f"F1 Score: {f1_score(y_test, predictions)}")
    print(f"AUC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])}")
    print(f"Confusion Matrix:\n {confusion_matrix(y_test, predictions)}")

    # Save the model
    joblib.dump(best_model, f'{endpoint}_svm_model.pkl')

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

import joblib  # For saving models

In [3]:
# this worked but i want more metrics
'''def train_dense_network(X_train, y_train, X_test, y_test):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for endpoint {endpoint}: {accuracy}")
    
    return model''';

In [20]:
def process_endpoint(endpoint, full_df):
    # Step 1: Filter rows with NaN in the target endpoint
    filtered_df = full_df.dropna(subset=[endpoint])

    # Step 2: Drop columns with NaN values
    clean_df = filtered_df.dropna(axis=1).copy()

    # Step 3: Define features excluding specific columns
    features = clean_df.columns.difference(['ID', 'inchikey', 'inchi', 'SMILES', 'sdftitle', 'order', 'set', 'inchi', endpoint])

    # Step 4 & 5: Define X and Y
    X = clean_df[features].values
    y = clean_df[endpoint].values

    # Step 6: Check shapes and assert they match
    assert X.shape[0] == y.shape[0], f"Shape mismatch between X and y for endpoint {endpoint}"

    # Step 7: Split into test/train and scale features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train dense network, now including the 'endpoint' argument
    model = train_dense_network(X_train_scaled, y_train, X_test_scaled, y_test, endpoint)

    # Save the model and scaler
    model.save(f'{endpoint}_dense_model.h5')
    joblib.dump(scaler, f'{endpoint}_scaler.pkl')


In [29]:
def train_dense_network(X_train, y_train, X_test, y_test, endpoint):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)
    
    # Generate predictions
    y_pred = model.predict(X_test)
    y_pred_class = (y_pred > 0.7).astype('int32') #push up the 0.5 to minimize false positives! this could be a slider if wanted

    # Calculate metrics
    precision = precision_score(y_test, y_pred_class)
    recall = recall_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class)
    auc = roc_auc_score(y_test, y_pred)

    # Print the evaluation metrics
    print(f"Evaluation Metrics for Endpoint '{endpoint}':")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC-AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_class)}")

    return model

In [28]:
# Example usage for one endpoint
endpoint = 'NR.ER'  # endpoint column name
process_endpoint(endpoint, df_ordered)

Epoch 1/50
173/173 [==============================] - 0s 814us/step - loss: 0.4231 - accuracy: 0.8567 - val_loss: 0.3414 - val_accuracy: 0.8894
Epoch 2/50
173/173 [==============================] - 0s 557us/step - loss: 0.3482 - accuracy: 0.8847 - val_loss: 0.3244 - val_accuracy: 0.8901
Epoch 3/50
173/173 [==============================] - 0s 554us/step - loss: 0.3210 - accuracy: 0.8950 - val_loss: 0.3215 - val_accuracy: 0.8944
Epoch 4/50
173/173 [==============================] - 0s 552us/step - loss: 0.3111 - accuracy: 0.8924 - val_loss: 0.3131 - val_accuracy: 0.8915
Epoch 5/50
173/173 [==============================] - 0s 550us/step - loss: 0.2994 - accuracy: 0.8997 - val_loss: 0.3098 - val_accuracy: 0.8930
Epoch 6/50
173/173 [==============================] - 0s 548us/step - loss: 0.2885 - accuracy: 0.9051 - val_loss: 0.3079 - val_accuracy: 0.8959
Epoch 7/50
173/173 [==============================] - 0s 557us/step - loss: 0.2842 - accuracy: 0.9051 - val_loss: 0.3075 - val_accuracy:

/Users/Ratthew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# increasing the certainty bar experiments

With determinator at 0.5:
Evaluation Metrics for Endpoint 'NR.ER':
Precision: 0.7684
Recall: 0.3967
F1 Score: 0.5233
ROC-AUC: 0.7630
Confusion Matrix:
[[1177   22]
 [ 111   73]]
 
@0.55
Precision: 0.7711
Recall: 0.3478
F1 Score: 0.4794
ROC-AUC: 0.7704
Confusion Matrix:
[[1180   19]
 [ 120   64]]
 
@0.65
Precision: 0.8429
Recall: 0.3207
F1 Score: 0.4646
ROC-AUC: 0.7650
Confusion Matrix:
[[1188   11]
 [ 125   59]]
 
@0.75
Precision: 0.9138
Recall: 0.2880
F1 Score: 0.4380
ROC-AUC: 0.7697
Confusion Matrix:
[[1194    5]
 [ 131   53]]
 

# loop all 12 at a decision threshold of 0.7

In [30]:
for endpoint in endpoints:
    print(f"Processing {endpoint}...")
    process_endpoint(endpoint, df_ordered)
    print(f"Finished processing {endpoint}.\n")

Processing NR.AhR...
Epoch 1/50
185/185 [==============================] - 0s 862us/step - loss: 0.3502 - accuracy: 0.8644 - val_loss: 0.2625 - val_accuracy: 0.9004
Epoch 2/50
185/185 [==============================] - 0s 575us/step - loss: 0.2548 - accuracy: 0.8983 - val_loss: 0.2474 - val_accuracy: 0.9024
Epoch 3/50
185/185 [==============================] - 0s 562us/step - loss: 0.2372 - accuracy: 0.9054 - val_loss: 0.2438 - val_accuracy: 0.9106
Epoch 4/50
185/185 [==============================] - 0s 559us/step - loss: 0.2256 - accuracy: 0.9051 - val_loss: 0.2435 - val_accuracy: 0.9106
Epoch 5/50
185/185 [==============================] - 0s 561us/step - loss: 0.2214 - accuracy: 0.9122 - val_loss: 0.2469 - val_accuracy: 0.9119
Epoch 6/50
185/185 [==============================] - 0s 557us/step - loss: 0.2012 - accuracy: 0.9163 - val_loss: 0.2439 - val_accuracy: 0.9079
Epoch 7/50
185/185 [==============================] - 0s 570us/step - loss: 0.2051 - accuracy: 0.9168 - val_loss: 0

/Users/Ratthew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
208/208 [==============================] - 0s 805us/step - loss: 0.1673 - accuracy: 0.9576 - val_loss: 0.1079 - val_accuracy: 0.9742
Epoch 2/50
208/208 [==============================] - 0s 646us/step - loss: 0.1269 - accuracy: 0.9702 - val_loss: 0.1075 - val_accuracy: 0.9754
Epoch 3/50
208/208 [==============================] - 0s 653us/step - loss: 0.1183 - accuracy: 0.9728 - val_loss: 0.1079 - val_accuracy: 0.9748
Epoch 4/50
208/208 [==============================] - 0s 675us/step - loss: 0.1141 - accuracy: 0.9745 - val_loss: 0.1065 - val_accuracy: 0.9754
Epoch 5/50
208/208 [==============================] - 0s 652us/step - loss: 0.1048 - accuracy: 0.9755 - val_loss: 0.1068 - val_accuracy: 0.9754
Epoch 6/50
208/208 [==============================] - 0s 648us/step - loss: 0.0998 - accuracy: 0.9766 - val_loss: 0.1125 - val_accuracy: 0.9748
Epoch 7/50
208/208 [==============================] - 0s 560us/step - loss: 0.0996 - accuracy: 0.9760 - val_loss: 0.1061 - val_accuracy:

/Users/Ratthew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
193/193 [==============================] - 0s 915us/step - loss: 0.1745 - accuracy: 0.9474 - val_loss: 0.0970 - val_accuracy: 0.9792
Epoch 2/50
193/193 [==============================] - 0s 597us/step - loss: 0.1018 - accuracy: 0.9726 - val_loss: 0.0955 - val_accuracy: 0.9773
Epoch 3/50
193/193 [==============================] - 0s 592us/step - loss: 0.0996 - accuracy: 0.9739 - val_loss: 0.0860 - val_accuracy: 0.9766
Epoch 4/50
193/193 [==============================] - 0s 602us/step - loss: 0.0929 - accuracy: 0.9750 - val_loss: 0.0856 - val_accuracy: 0.9779
Epoch 5/50
193/193 [==============================] - 0s 594us/step - loss: 0.0766 - accuracy: 0.9758 - val_loss: 0.0849 - val_accuracy: 0.9773
Epoch 6/50
193/193 [==============================] - 0s 566us/step - loss: 0.0743 - accuracy: 0.9799 - val_loss: 0.0845 - val_accuracy: 0.9773
Epoch 7/50
193/193 [==============================] - 0s 583us/step - loss: 0.0704 - accuracy: 0.9792 - val_loss: 0.0846 - val_accuracy:

...

/Users/Ratthew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
167/167 [==============================] - 0s 1ms/step - loss: 0.4089 - accuracy: 0.8383 - val_loss: 24.8807 - val_accuracy: 0.8776
Epoch 2/50
167/167 [==============================] - 0s 621us/step - loss: 0.3074 - accuracy: 0.8697 - val_loss: 17.6274 - val_accuracy: 0.8851
Epoch 3/50
167/167 [==============================] - 0s 633us/step - loss: 0.2642 - accuracy: 0.8916 - val_loss: 12.9359 - val_accuracy: 0.8941
Epoch 4/50
167/167 [==============================] - 0s 690us/step - loss: 0.2565 - accuracy: 0.8922 - val_loss: 9.5678 - val_accuracy: 0.9032
Epoch 5/50
167/167 [==============================] - 0s 669us/step - loss: 0.2339 - accuracy: 0.9022 - val_loss: 10.5479 - val_accuracy: 0.9047
Epoch 6/50
167/167 [==============================] - 0s 676us/step - loss: 0.2282 - accuracy: 0.9061 - val_loss: 7.4427 - val_accuracy: 0.9062
Epoch 7/50
167/167 [==============================] - 0s 584us/step - loss: 0.2153 - accuracy: 0.9097 - val_loss: 11.7124 - val_accura

/Users/Ratthew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Finished processing SR.MMP.

Processing SR.p53...
Epoch 1/50
195/195 [==============================] - 0s 819us/step - loss: 0.2732 - accuracy: 0.9245 - val_loss: 0.2066 - val_accuracy: 0.9325
Epoch 2/50
195/195 [==============================] - 0s 598us/step - loss: 0.2040 - accuracy: 0.9378 - val_loss: 0.1951 - val_accuracy: 0.9312
Epoch 3/50
195/195 [==============================] - 0s 603us/step - loss: 0.1792 - accuracy: 0.9397 - val_loss: 0.1810 - val_accuracy: 0.9364
Epoch 4/50
195/195 [==============================] - 0s 592us/step - loss: 0.1650 - accuracy: 0.9447 - val_loss: 0.1777 - val_accuracy: 0.9370
Epoch 5/50
195/195 [==============================] - 0s 611us/step - loss: 0.1521 - accuracy: 0.9470 - val_loss: 0.1733 - val_accuracy: 0.9364
Epoch 6/50
195/195 [==============================] - 0s 536us/step - loss: 0.1520 - accuracy: 0.9474 - val_loss: 0.1716 - val_accuracy: 0.9370
Epoch 7/50
195/195 [==============================] - 0s 584us/step - loss: 0.1417 - a

/Users/Ratthew/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
